In [1]:
%load_ext autoreload
%autoreload 2

import requests
import sqlite3
import time

from bs4 import BeautifulSoup
from googlesearch import search
from kerykeion import (
    AstrologicalSubject,
    Houses,
    KerykeionPointModel,
    Planet,
    PointType,
    Report,
    ZodiacSignModel,
)
from pydantic import computed_field
from readability import Document
from typing import get_args
from typing import Literal, Union

## Get the astrological birth chart

In [ ]:
subject = AstrologicalSubject(
    name="Subject",
    year=1996, month=3, day=20,
    hour=17, minute=33,
    lng=72.88261, lat=19.07283,
    tz_str="Asia/Kolkata",
    city="Mumbai",
    zodiac_type="Sidereal",
    sidereal_mode="LAHIRI",
)
report = Report(subject)
report.print_report()

In [3]:
Nakshatra = Literal[
    "Ashwini", "Bharani", "Krittika", "Rohini", "Mrigashira", "Ardra", "Punarvasu", "Pushya", "Ashlesha",
    "Magha", "Purva Phalguni", "Uttara Phalguni", "Hasta", "Chitra", "Swati", "Vishakha", "Anuradha", "Jyeshtha",
    "Mula", "Purva Ashadha", "Uttara Ashadha", "Shravana", "Dhanishta", "Shatabhisha", "Purva Bhadrapada",
    "Uttara Bhadrapada", "Revati"
]
Pada = Literal[1, 2, 3, 4]

class CustomKerykeionPointModel(KerykeionPointModel):
    name: Union[Planet, Houses, Literal["South_Node"]]

    @computed_field(return_type=Nakshatra)
    def nakshatra(self) -> Nakshatra:
        index = int(self.abs_pos // 13.3333)
        return get_args(Nakshatra)[index]

    @computed_field(return_type=Pada)
    def pada(self):
        index = int((self.abs_pos % 13.3333) // 3.3333)
        return get_args(Pada)[index]

    @classmethod
    def get_kerykeion_point_from_degree(cls, degree: Union[int, float], name: str, point_type: PointType) -> str:
        ZODIAC_SIGNS = {
            0: ZodiacSignModel(sign="Ari", quality="Cardinal", element="Fire", emoji="♈️", sign_num=0),
            1: ZodiacSignModel(sign="Tau", quality="Fixed", element="Earth", emoji="♉️", sign_num=1),
            2: ZodiacSignModel(sign="Gem", quality="Mutable", element="Air", emoji="♊️", sign_num=2),
            3: ZodiacSignModel(sign="Can", quality="Cardinal", element="Water", emoji="♋️", sign_num=3),
            4: ZodiacSignModel(sign="Leo", quality="Fixed", element="Fire", emoji="♌️", sign_num=4),
            5: ZodiacSignModel(sign="Vir", quality="Mutable", element="Earth", emoji="♍️", sign_num=5),
            6: ZodiacSignModel(sign="Lib", quality="Cardinal", element="Air", emoji="♎️", sign_num=6),
            7: ZodiacSignModel(sign="Sco", quality="Fixed", element="Water", emoji="♏️", sign_num=7),
            8: ZodiacSignModel(sign="Sag", quality="Mutable", element="Fire", emoji="♐️", sign_num=8),
            9: ZodiacSignModel(sign="Cap", quality="Cardinal", element="Earth", emoji="♑️", sign_num=9),
            10: ZodiacSignModel(sign="Aqu", quality="Fixed", element="Air", emoji="♒️", sign_num=10),
            11: ZodiacSignModel(sign="Pis", quality="Mutable", element="Water", emoji="♓️", sign_num=11),
        }
        sign_index = int(degree // 30)
        sign_degree = degree % 30
        zodiac_sign = ZODIAC_SIGNS[sign_index]
        return CustomKerykeionPointModel(
            name=name,
            quality=zodiac_sign.quality,
            element=zodiac_sign.element,
            sign=zodiac_sign.sign,
            sign_num=zodiac_sign.sign_num,
            position=sign_degree,
            abs_pos=degree,
            emoji=zodiac_sign.emoji,
            point_type=point_type,
        )

    @classmethod
    def copy_from(cls, source: KerykeionPointModel) -> 'CustomKerykeionPointModel':
        return cls(
            name=source.name,
            quality=source.quality,
            element=source.element,
            sign=source.sign,
            sign_num=source.sign_num,
            position=source.position,
            abs_pos=source.abs_pos,
            emoji=source.emoji,
            point_type=source.point_type,
            house=source.house,
            retrograde=source.retrograde,
        )

In [ ]:
planets_list = [subject.sun, subject.moon, subject.mercury, subject.venus, subject.mars, subject.jupiter, subject.saturn, subject.mean_node]
planets_list = [CustomKerykeionPointModel.copy_from(planet) for planet in planets_list]
south_node = CustomKerykeionPointModel.get_kerykeion_point_from_degree(
    (subject.mean_node.abs_pos + 180) % 360,
    "South_Node",
    point_type=subject.mean_node.point_type,
)
planets_list.append(south_node)
for planet in planets_list:
    print(f"{planet.name} is in {planet.nakshatra}, {planet.pada}")

## Get House lordships

In [5]:
LORDS = {
    0: "Mars",
    1: "Venus",
    2: "Mercury",
    3: "Moon",
    4: "Sun",
    5: "Mercury",
    6: "Venus",
    7: "Mars",
    8: "Jupiter",
    9: "Saturn",
    10: "Saturn",
    11: "Jupiter",
}
queries = []
for house in subject._houses_list:
    lord = LORDS[house.sign_num]
    lord = next(planet for planet in planets_list if planet.name == lord)
    queries.append(f"{lord.name} is in {lord.house}")
    queries.append(f"{lord.name} is in {lord.nakshatra} Pada {lord.pada}")
    queries.append(f"{house.name} lord is in {lord.house}")

## Google search relevant documents

In [ ]:
urls = []
pause_per_query = 3
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
for query in queries:
	query += " site:astrosaxena.in"
	for j in search(query, tld="co.in", stop=3, pause=pause_per_query, user_agent=user_agent):
		urls.append(j)
	time.sleep(pause_per_query)

In [ ]:
query

You are a super smart vedic astrologer. You are given birth chart placements and have to answer the following questions:

1. What are this individuals biggest strengths?
2. What is something this individual is going to struggle with a lot in life? Connect this back to a real medical or psychological diagnosis.

Reason why on each point.

First House = Leo Ascendant (ruled by Sun)
Second House = Rahu in Virgo (ruled by Mercury)
Third House = Libra (ruled by Venus)
Fourth House = Scorpio (ruled by Mars)
Fifth House = Jupiter in Sagittarius (ruled by Jupiter)
Sixth House = Capricorn (ruled by Saturn)
Seventh House = Mercury in Aquarius (ruled by Saturn) and Mars, Saturn and Sun in Pisces (ruled by Jupiter)
Eighth House = Moon and Ketu, i.e. South Node in Pisces (ruled by Jupiter)
Ninth House = Venus in Aries (ruled by Mars)
Tenth House = Taurus (ruled by Venus)
Eleventh House = Gemini (ruled by Mercury)
Twelfth House = Cancer (ruled by Moon)

First House = Virgo Ascendant (ruled by Mercury)
Second House = Venus and Jupiter in Libra (ruled by Venus)
Third House = Scorpio (ruled by Mars)
Fourth House = Moon in Sagittarius (ruled by Jupiter)
Fifth House = Capricorn (ruled by Saturn)
Sixth House = Ketu in Aquarius (ruled by Saturn)
Seventh House = Pisces (ruled by Jupiter)
Eighth House = Aries (ruled by Mars)
Ninth House = Taurus (ruled by Venus)
Tenth House = Mercury in Gemini (ruled by Mercury)
Eleventh House = Mars in Cancer (ruled by Moon)
Twelfth House = Sun, Saturn and Rahu, i.e. North node in Leo (ruled by Sun)

## Extract content from internet documents

In [ ]:
# def get_url_content(url: str):
#     response = requests.get(url, headers={'User-Agent': user_agent})
#     doc = Document(response.content)
#     soup = BeautifulSoup(doc.summary(), 'html.parser')
#     text = soup.get_text()
#     return doc.title(), text

# get_url_content("https://www.astrosaxena.in/astrology/info/703/concept-of-combust-planets")

In [ ]:
# create_table_sql = """
# create table if not exists knowledge_base (
#     id INTEGER primary key autoincrement,
#     url TEXT not null unique,
#     content TEXT not null,
#     created_at TIMESTAMP not null default CURRENT_TIMESTAMP,
# );
# """
# connection = sqlite3.connect("astrosaxena.db")
# cursor = connection.cursor()
# cursor.execute(create_table_sql)
# try:
#     for url in urls:
#         text = get_url_content(url)
#         cursor.execute("insert into knowledge_base values (url, content)")
#         connection.commit()
# finally:
#     connection.close()

## Retrieval-augmented answering